# Tutorial about how to use a standard Analysis class

In [ ]:
from pathlib import Path

%matplotlib inline

import matplotlib.pyplot as plt

import locan as lc

In [ ]:
lc.show_versions(system=False, dependencies=False, verbose=False)

## Load rapidSTORM data file

Identify some data in the test_data directory and provide a path using pathlib.Path

In [ ]:
path = lc.ROOT_DIR / 'tests/test_data/rapidSTORM_dstorm_data.txt'
print(path, '\n')
dat = lc.load_rapidSTORM_file(path=path, nrows=1000)

In [ ]:
print(dat.data.head(), '\n')
print('Summary:')
dat.print_summary()
print('Properties:')
print(dat.properties)

## Visualization

For visualizing the data use one of the rendering methods.

In [ ]:
lc.render_2d_mpl(dat, bin_size=100, rescale='equal');

## A simple analysis procedure: localization precision

### Instantiation

Create an instance of the analysis class. By doing this you set all parameters in the parameter attribute. To start the actual computation you have to call instance.compute().

In [ ]:
lp = lc.LocalizationPrecision(radius=50)

Each analysis class provides some attributes and methods for the most common interactions with the computed results.

In [ ]:
attributes = [x for x in dir(lp) if not x.startswith('_')]
attributes

### The results attribute

A standard analysis class has an attribute *results* to hold the most fundamental results as number, numpy array or pandas series or dataframe. 

In [ ]:
lp.compute(dat)
print('type of lpf.results: ', type(lp.results), '\n')
print(lp.results.head())

Results can be saved by specifying a path

In [ ]:
from pathlib import Path
temp_directory = Path('.') / 'temp'
temp_directory.mkdir(parents=True, exist_ok=True)

path = temp_directory / 'results.txt'
path

and saving the data using numpy or pandas routines

In [ ]:
lp.results.to_csv(path, sep='\t')

Delete the file and empty directory

In [ ]:
path.unlink()
temp_directory.rmdir()

### A simple standardized plot

Most likely the results should be inspected by looking at a typical plot.
In this case the plot shows results smoothed by a running average according to the specified window.

In [ ]:
lp.plot(window=10);

For more advanced plotting schemes use the matplotlip framework.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,3))
lp.plot(ax=ax[0], window=10, loc_property='position_delta_x')
lp.plot(ax=ax[1], window=10, loc_property='position_delta_y')
plt.tight_layout()
plt.show()

### A simple standardized histogram

Quite often the results are best presented as a histogram. The histogram for the distances per default includes a fit to a distribution expected for normal distributed localizations. Sigma is the localization precision.

The histogram per default provides automatic bins and is normalized to show a probability density function.

In [ ]:
lp.hist(loc_property='position_delta_x');

Alternatively the position deltas can be histogrammed.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)
lp.hist(ax=ax[0], loc_property='position_delta_x')
lp.hist(ax=ax[1], loc_property='position_delta_y')
plt.tight_layout()
plt.show()

### Secondary results 

Secondary results are e.g. fit parameter derived from analyzing the distribution of *results* values. Secondary results are different for each analysis routine. They are included as additional attributes.

Localization precision can e.g. be derived from fitting the position distances to an appropriate distribution and estimating the sigma parameter.

In [ ]:
lp.distribution_statistics.parameter_dict()

In [ ]:
print('position_distance_sigma: ', lp.distribution_statistics.parameter_dict()['position_distance_sigma'])

### Metadata

Each analysis class is supplied with meta data. The main purpose is to (i) capture methods and parameters that were supplied in each instantiation and (ii) provide information on the dataset on which the particular analysis was carried out. Metadata is structured using protocol buffers.

In [ ]:
lp.meta

You can add some user-defined key-value pairs:

In [ ]:
lp.meta.map['some key'] = 'some value'
lp.meta.map

## Metadata can be used to rerun the analysis with the same parameter.

In [ ]:
lp.meta.method.name

In [ ]:
lp.meta.method.parameter

In [ ]:
import ast
import locan.analysis
params = ast.literal_eval(lp.meta.method.parameter)
print(params)
lp_2 = getattr(locan.analysis, lp.meta.method.name)(**params)
lp_2.compute(dat)
lp_2.results.head()